# Maximum likelihood via gradient ascent
In this task we generate data form a univariate Gaussian distribution with unknown mean $\mu$ and precision $\tau$. We want to estimate the maximum likelihood parameters using an iterative greedy procedure. At each move we calculate the derivative of the log likelihood of a data batch at the current location in parameter space, then move a distance in that direction, and go again

We will try several different combinations here. 
* Gradients
   * Eucledian
   * Natural gradients
* Batch size
   * Gradients based on the full sample
   * Gradients based on a "mini-batch" of 1 sample.
   
We would hope to see a little bit about the effect of making the different choices.

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

### Helper functions

**Data batch generator**

Define a generator by `batch_generator = data_generator(data_set, batch_size)`, and follow that up by `sample = next(batch_generator)`. 

In [ ]:
def data_generator(array, batch_size):
    """Generate batch with respect to array's first axis."""
    start = 0  # pointer to where we are in iteration
    while True:
        stop = start + batch_size
        diff = stop - array.shape[0]
        if diff <= 0:
            batch = array[start:stop]
            start += batch_size
        else:
            batch = np.concatenate((array[start:], array[:diff]))
            start = diff
        yield batch

**Function to adjust the learning-rate**

In [ ]:
def calculate_learning_rate(_step, _start_lr=1e-3, _divisor=100, _exponent=1.5):
    lr_scale = (1. + _step / _divisor) ** (-_exponent)
    return lr_scale * _start_lr

**Plot generator**

Function to generate a plot. Lots of freedom here -- what data to use to get a likelihood contour, should we show the trace through the parameter landscape, should we add the empirical ML estimates calculated in closed form, should we have the gradients indicated in the plot, etc.

In [ ]:
def generate_plot(data_for_likelihhood_contour, full_data=None, mu_list=None, tau_list=None,
                  gradient_values=None,
                  current_mu=None, current_tau=None, title=None):
    plt.clf()
    plt.cla()
    plt.close('all')

    # Generate a contour-plot using the data se nt in. This will either be a batch or the full dataset
    mu_mesh, tau_mesh = np.meshgrid(np.linspace(-2, 2, 100),
                                    np.linspace(1E-6, 2, 100))
    log_likelihood = np.zeros_like(mu_mesh)
    for i in range(len(data_for_likelihhood_contour)):
        log_likelihood += norm.logpdf(data_for_likelihhood_contour[i], loc=mu_mesh, scale=1. / np.sqrt(tau_mesh))
    plt.contour(mu_mesh, tau_mesh, log_likelihood, 100)

    # If the function receives values for mu_list and tau_list, then plot the trace
    if mu_list is not None and tau_list is not None:
        for point in range(step):
            plt.plot(mu_list[point:point + 2], tau_list[point:point + 2], 'g--')

    # Add gradient vector to plot if we have the info
    if gradient_values is not None:
        # Normalize for visual reasons
        scale = .25 / np.sqrt(np.sum(np.square(gradient_values)))
        plt.gca()
        plt.plot(
            [current_mu, current_mu + scale * gradient_values[0]],
            [current_tau, current_tau + scale * gradient_values[1]],
            'b-')

    # Add point before update
    if current_mu is not None and current_tau is not None:
        plt.gca()
        plt.plot(current_mu, current_tau, 'ro')

    # Add max likelihood params
    if full_data is not None:
        best_mean = np.mean(full_data)
        best_prec = 1. / np.cov(full_data)
        plt.plot(best_mean, best_prec, 'k*')

    # Pimp up the figure
    plt.xlabel('Expectation $\mu$')
    plt.ylabel('Precision $\\tau$')
    if title is not None:
        plt.title(title)

    plt.show()

**Calculate log likelihood**

Calculate the log likelihood of a solution (`_mu`, `_tau`). This is quite straight forward, it is simply the log likelihood of a Gaussian with known parameters

In [ ]:
def calculate_log_likelihood(_data, _mu, _tau):
    return np.sum(norm(loc=_mu, scale=1./np.sqrt(_tau)).logpdf(_data))

## Calculate gradients  

This function taeks as input the data-batch `_data` and the location (`_mu`, `_tau`) at  which the gradient is to be calculated. The function returns two lists, the natural gradients (wrt mu first, then wrt. tau) and the euclidean gradients (same order mu, tau).

In [ ]:
def calculate_gradient(N, _data, _mu, _tau):
    partial_mu = _tau * np.sum(_data - _mu)
    partial_tau = .5 * len(_data) / _tau - .5 * np.sum(np.square(_data - _mu))

    euclidean_gradient = N / _data.shape[0] * np.array([partial_mu, partial_tau])

    # Multiply by inverse Fisher matrix. Fisher matrix is
    # diagonal(tau, .5 / tau**2) so inverse is diag( 1/tau, 2 * tau**2 )
    partial_mu = partial_mu / _tau
    partial_tau = partial_tau * 2 * _tau * _tau

    natural_gradient = np.array([partial_mu, partial_tau])

    return natural_gradient, euclidean_gradient

## Setup. 
## The important things here are `batch_size` and `do_natural_gradient`.
## Play around with both of them

In [ ]:
# Set the batch size. 
# This should either be 1 (mini-batching) or everything (full_data.shape[0])
batch_size = 1    # Alternatively full_data.shape[0]

# Set parameters
do_natural_gradient = False

# Sample data
np.random.seed(123)
full_data = np.random.randn(100)
batch_generator = data_generator(full_data, batch_size)

# Set starting position
current_mu = -1
current_tau = .1

# Set the initial learning rate
initial_learning_rate = 1E-3

# Store the moves made and the log_likelihood of each intermediuate solution in a list. Used for plotting
mu_list = [current_mu]
tau_list = [current_tau]
total_examples_seen = [0]
log_likelihoods = [calculate_log_likelihood(_data=full_data, _mu=current_mu, _tau=current_tau)]

# Label-text
pic_text = 'Natural' if do_natural_gradient is True else 'Euclidian'

## Do the iterations

In [ ]:
for step in range(101):
    learning_rate = calculate_learning_rate(_step=step, _start_lr=initial_learning_rate, _exponent=1.1)
    sample = next(batch_generator)
    nat_grad, euc_grad = calculate_gradient(N=full_data.shape[0], _data=sample, _mu=current_mu, _tau=current_tau)
    grad = nat_grad if do_natural_gradient is True else euc_grad

    if (step % 25) == 0:
        generate_plot(data_for_likelihhood_contour=sample, full_data=full_data,
                      mu_list=mu_list, tau_list=tau_list,
                      gradient_values=grad,
                      current_mu=current_mu, current_tau=current_tau,
                      title='{:s} gradients; batch={:d}; iter {:d}'.format(pic_text, batch_size, step)
                     )

    current_mu += learning_rate * grad[0]
    current_tau += learning_rate * grad[1]

    mu_list.append(current_mu)
    tau_list.append(current_tau)
    
    total_examples_seen.append(batch_size + total_examples_seen[-1])
    log_likelihoods.append(calculate_log_likelihood(_data=full_data, _mu=current_mu, _tau=current_tau))


## Show off with the final results

In [ ]:
# We are done. Show the final plot, where the empiricalø log likelihood profile of the full dataset
# is shown together with the movements, the final location (that is random if we do small-batch), and
# the empirical max likelihood parameters.
generate_plot(data_for_likelihhood_contour=full_data, full_data=full_data,
              mu_list=mu_list, tau_list=tau_list,
              gradient_values=None,
              current_mu=current_mu, current_tau=current_tau,
              title ='{:s} gradients; batch={:d}; Done'.format(pic_text, batch_size))

In [ ]:
plt.plot(total_examples_seen, log_likelihoods)
plt.grid(True)
plt.title("Log likelihood as a function of examples seen")
plt.xlabel("Number of examples seen")
plt.ylabel("Log likelihood")
plt.xlim([0, total_examples_seen[-1] - 1])
plt.show()
